In [1]:
import cornac
from cornac.data import Reader, Dataset
from recommenders.evaluation.python_evaluation import map, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
import pandas as pd

In [2]:
reader = Reader(bin_threshold=0)
train = pd.read_csv("../datasets/ml-latest-small/stratified_split" + '''/train.csv''', sep=',', dtype='object')
test = pd.read_csv("../datasets/ml-latest-small/stratified_split" + '''/test.csv''', sep=',', dtype='object')
train_set = Dataset.from_uirt(reader.read("../datasets/ml-latest-small/stratified_split" + '''/train.csv''', sep=',', skip_lines=1, fmt="UIRT"))
test_set = Dataset.from_uirt(reader.read("../datasets/ml-latest-small/stratified_split" + '''/test.csv''', sep=',', skip_lines=1, fmt="UIRT"))

In [3]:
bpr = cornac.models.BPR(k=200, max_iter=200, learning_rate=0.01, lambda_reg=1e-3, seed=42, verbose=True)

In [4]:
bpr.fit(train_set=train_set)

  0%|          | 0/200 [00:00<?, ?it/s]

Optimization finished!


In [5]:
all_predictions = predict_ranking(bpr, train, usercol='userId', itemcol='movieId', predcol='prediction', remove_seen=True)
all_predictions

,userId,movieId,prediction
1,1,10,5.828436
2,1,100,1.077691
3,1,100044,-1.850520
4,1,100068,-0.597487
5,1,100083,-2.710454
...,...,...,...
5380195,99,99853,-0.555906
5380196,99,999,1.657978
5380197,99,99910,-1.207250
5380198,99,99917,-0.417304


In [6]:
k_list = [5, 10, 20, 30, 50, 100]
for k in k_list:
    eval_ndcg = ndcg_at_k(test, all_predictions, col_user='userId',
        col_item='movieId',
        col_prediction='prediction', k=k)

    print(f'''NDCG@{k}: {eval_ndcg}''')

NDCG@5: 0.2771635087571633
NDCG@10: 0.26402331482342045
NDCG@20: 0.2586989322310322
NDCG@30: 0.2645775324719106
NDCG@50: 0.2837745895461244
NDCG@100: 0.3191461225806324


In [7]:
k = 10
eval_map = map(test, all_predictions, col_user='userId',
    col_item='movieId',
    col_prediction='prediction', k=k)
eval_precision = precision_at_k(test, all_predictions, col_user='userId',
    col_item='movieId', col_prediction='prediction', k=k)
eval_recall = recall_at_k(test, all_predictions, col_user='userId',
    col_item='movieId', col_prediction='prediction', k=k)
eval_ndcg = ndcg_at_k(test, all_predictions, col_user='userId',
    col_item='movieId',
    col_prediction='prediction', k=k)

print(f'''NDCG@{k}: {eval_ndcg}''')
print(f'''MAP@{k}: {eval_map}''')
print(f'''Precision@{k}: {eval_precision}''')
print(f'''Recall@{k}: {eval_recall}''')

NDCG@10: 0.26402331482342045
MAP@10: 0.06652183905982624
Precision@10: 0.23770491803278687
Recall@10: 0.11925884665384252
